#### Loads dev.in as a list, and separates each sentence

In [21]:
def load_original_train():
    with open("train", encoding="utf-8") as file:
        train_list = file.readlines()
        train_list = [x.strip() for x in train_list]
        return train_list
    
train_list = load_original_train()

In [22]:
annotationDict = {"O": 0, "I-positive": 0, "B-positive": 0, "I-negative":0, "B-negative": 0, "I-neutral":0, "B-neutral":0}

for i in range(len(train_list)):
    annotated_word = train_list[i]
    each_word = annotated_word.split(" ")
    if (len(each_word) == 2):
        if each_word[1] == "O":
            annotationDict["O"] += 1
        elif each_word[1] == "I-positive":
            annotationDict["I-positive"] += 1
        elif each_word[1] == "B-positive":
            annotationDict["B-positive"] += 1
        elif each_word[1] == "I-negative":
            annotationDict["I-negative"] += 1
        elif each_word[1] == "B-negative":
            annotationDict["B-negative"] += 1
        elif each_word[1] == "I-neutral":
            annotationDict["I-neutral"] += 1
        elif each_word[1] == "B-neutral":
            annotationDict["B-neutral"] += 1

print(annotationDict)

{'O': 24242, 'I-positive': 607, 'B-positive': 1208, 'I-negative': 133, 'B-negative': 382, 'I-neutral': 23, 'B-neutral': 65}


In [33]:
def load_files():
    with open("dev.in", encoding="utf-8") as file:
        dev_in_list = file.readlines()
        dev_in_list = [x.strip() for x in dev_in_list]
        return dev_in_list
    
def load_modified_train_file():
    with open("modified_train", encoding="utf-8") as file:
        modified_train_list = file.readlines()
        modified_train_list = [x.strip() for x in modified_train_list]
        return modified_train_list
        
def split_into_sentences(dev_in_list):
    sentences = []
    each_sentence = []
    for word in dev_in_list:
        if word == "":
            sentences.append(each_sentence)
            each_sentence = []
        else:
            each_sentence.append(word)
        
    return sentences

modified_train_list = load_modified_train_file()   
dev_in_list = load_files()
sentences = split_into_sentences(dev_in_list)

#### Giant Emission and Giant Transition Dict

In [84]:
def count_word_tag_pair_dict():
    word_tag_pair_dict= {}
    for word_tag_pair in modified_train_list:
        if word_tag_pair_dict.get(word_tag_pair) == None:
            word_tag_pair_dict[word_tag_pair] = 1
        else:
            word_tag_pair_dict[word_tag_pair] += 1
    return word_tag_pair_dict

def get_word_dict():
    modifiedWordDict = {}        
    for i in range(len(modified_train_list)):
        annotated_word = modified_train_list[i]
        each_word = annotated_word.split(" ")
        if each_word[0] in modifiedWordDict:
            modifiedWordDict[each_word[0]] += 1
        else:
            modifiedWordDict[each_word[0]] = 1

    return modifiedWordDict
    

def store_estimate_emission_fix():
    giantEmissionDict = {}
    for word_tag_pair in modified_train_list:
        if word_tag_pair != "":
            word_tag_pair_list = word_tag_pair.split(" ")
            countTop = wordTagPairDict[word_tag_pair]
            countBottom = annotationDict[word_tag_pair_list[1]]
            emission = countTop/countBottom
            if giantEmissionDict.get(word_tag_pair) == None:
                giantEmissionDict[word_tag_pair] = countTop/countBottom
    return giantEmissionDict

def store_estimate_transition():
    giantTransitionDict = {}
    finalGiantTransitionDict = {}
    for taggedEachTweet in giantTaggedEachTweet:
        for i in range(len(taggedEachTweet) - 1):
            if giantTransitionDict.get(taggedEachTweet[i]+" "+taggedEachTweet[i+1]) == None:
                giantTransitionDict[taggedEachTweet[i]+" "+taggedEachTweet[i+1]] = 1
            else:
                giantTransitionDict[taggedEachTweet[i]+" "+taggedEachTweet[i+1]] += 1
    for eachTransition in giantTransitionDict:
        eachTransitionList = eachTransition.split(" ")
        startState = eachTransitionList[0]
        countTop = giantTransitionDict[eachTransition]
        countBottom = totalStateNumber[startState]
        transitionParam = countTop/countBottom
        finalGiantTransitionDict[eachTransition] = transitionParam
        
    return finalGiantTransitionDict

def separate_tweets():
    countStart = 1
    # counting starts
    giant_each_tweet = []
    each_tweet = []
    totalStateNumber = {}
    for i in range(len(modified_train_list)):
        annotated_word = modified_train_list[i]
        each_word = annotated_word.split(" ")
        if each_word == ['']:
            countStart += 1
            giant_each_tweet.append(each_tweet)
            each_tweet = []

        else:
            each_tweet.append(each_word[1])
    for i in range(len(giant_each_tweet)):
        giant_each_tweet[i].insert(0, 'START')
        giant_each_tweet[i].append('STOP')
    
    for each_tweet in giant_each_tweet:
        for tag in each_tweet:
            if totalStateNumber.get(tag) == None:
                totalStateNumber[tag] = 1
            else:
                totalStateNumber[tag] += 1

    return giant_each_tweet, countStart, totalStateNumber

modifiedWordDict = get_word_dict()
wordTagPairDict = count_word_tag_pair_dict()
giantEmissionDict = store_estimate_emission_fix()
giantTaggedEachTweet, countStart, totalStateNumber = separate_tweets()
giantTransitionDict = store_estimate_transition()
print(giantTransitionDict["START O"])
# print(giantTransitionDict)
# print(len(giantTransitionDict))

0.9401709401709402


In [68]:
def forward_prob(i,u, sentence):
    all_tags=["O","B-negative","B-positive","B-neutral","I-negative","I-positive","I-neutral"]
    last_word = sentence[len(sentence)-1]
    # base case
    if i == 1:
        if giantTransitionDict.get("START" + " " + u) != None:
            a = giantTransitionDict["START" + " " + u]#transition from start to u
            return a
        else:
#             print(u)
            return 0
    # recursive case
    else:
        word_at_u = sentence[i-1]
#         print(word_at_u)
        if word_at_u in modifiedWordDict:
            word_at_u = word_at_u
        else:
            word_at_u = "#UNK#"
        forward_prob_list = []
        for v in all_tags:
            if giantTransitionDict.get(v + " "+ u) != None and giantEmissionDict.get(word_at_u+" "+v ) != None:
                a = giantTransitionDict[v +" "+ u]#transition from v to u #read from dict
                b = giantEmissionDict[word_at_u + " "+ v]#emission from v to xi #read from dict
                forward_prob_from_v = forward_prob(i-1, v, sentence)*a*b
                forward_prob_list.append(forward_prob_from_v)
        return sum(forward_prob_list)
    
def back_prob(i,u, sentence):
    all_tags=["O","B-negative","B-positive","B-neutral","I-negative","I-positive","I-neutral"]
    last_word = sentence[len(sentence)-1]
    n = len(sentence)
    # base case
    if i == n:
        if giantTransitionDict.get(u + " " + "STOP") != None and giantEmissionDict.get(last_word + " " + u) != None:
            a = giantTransitionDict[u + " " + "STOP"]#transition from u to stop
            b = giantEmissionDict[last_word + " " + u]#emission from u to xn
            return a*b
        else:
            return 0
    # recursive case
    else:
        word_at_v = sentence[i]
        if word_at_v in modifiedWordDict:
            word_at_v = word_at_v
        else:
            word_at_v = "#UNK#"
        back_prob_list = []
        for v in all_tags:
            if i == 1:
                v = "START"
            if giantTransitionDict.get(u + " "+ v) != None and giantEmissionDict.get(word_at_v + " " + u) != None:
                a = giantTransitionDict[u + " " + v]#transition from u to v
                b = giantEmissionDict[word_at_v + " " + u]#emission from u to xi
                back_prob_from_v = back_prob(i + 1, v, sentence) * a * b
                back_prob_list.append(back_prob_from_v)
#                 print(back_prob_list)

        return sum(back_prob_list)


In [69]:
def combine_path(best_path, sentence):
#     print(best_path)
#     print(sentence)
#     print(len(best_path))
#     print(len(sentence))
    result = ""
    for i in range(len(sentence)):
        if (sentence[i] == ''):
            result += " \n"
        else:
            result += sentence[i] + " " + best_path[i] + "\n"

    return result


## Max Marginal Decoding Try 1

In [70]:
def max_marginal(sentence):
    sentence.append("")
    #sentence is a list containing the words
    # this is only for one sentence
    opti_path = []
    all_tags=["O","B-negative","B-positive","B-neutral","I-negative","I-positive","I-neutral"]
    for i in range(len(sentence)-1): #minus 1, so you dont consider ""
        opti_y_dict = {}
        for tag in all_tags:
            print(tag)
            f = forward_prob(i,tag, sentence)
            b = back_prob(i,tag, sentence)
            opti_y_at_i = f*b
            opti_y_dict[tag] = opti_y_at_i
            print(opti_y_dict)
        max_i_path = max(opti_y_dict, key=opti_y_dict.get)

        opti_path.append(max_i_path)
    return opti_path

shortSentence = ['The', 'tuna', 'and', '#UNK#', 'potatoes', 'are', 'excellent', '.']
max_marginal(shortSentence)

O
{'O': 0.0}
B-negative
{'O': 0.0, 'B-negative': 0.0}
B-positive
{'O': 0.0, 'B-negative': 0.0, 'B-positive': 0.0}
B-neutral
{'O': 0.0, 'B-negative': 0.0, 'B-positive': 0.0, 'B-neutral': 0.0}
I-negative
{'O': 0.0, 'B-negative': 0.0, 'B-positive': 0.0, 'B-neutral': 0.0, 'I-negative': 0}
I-positive
{'O': 0.0, 'B-negative': 0.0, 'B-positive': 0.0, 'B-neutral': 0.0, 'I-negative': 0, 'I-positive': 0}
I-neutral
{'O': 0.0, 'B-negative': 0.0, 'B-positive': 0.0, 'B-neutral': 0.0, 'I-negative': 0, 'I-positive': 0, 'I-neutral': 0}
O
{'O': 0.0}
B-negative
{'O': 0.0, 'B-negative': 0.0}
B-positive
{'O': 0.0, 'B-negative': 0.0, 'B-positive': 0.0}
B-neutral
{'O': 0.0, 'B-negative': 0.0, 'B-positive': 0.0, 'B-neutral': 0.0}
I-negative
{'O': 0.0, 'B-negative': 0.0, 'B-positive': 0.0, 'B-neutral': 0.0, 'I-negative': 0}
I-positive
{'O': 0.0, 'B-negative': 0.0, 'B-positive': 0.0, 'B-neutral': 0.0, 'I-negative': 0, 'I-positive': 0}
I-neutral
{'O': 0.0, 'B-negative': 0.0, 'B-positive': 0.0, 'B-neutral': 0.0, 

['O', 'O', 'O', 'O', 'O', 'O', 'O', 'O']

## Max Marginal Decoding By Zanette

In [ ]:
def max_marginal(td,transition,emission,alpha_base):
    print("Doing max marginal...")
    sentences = split_sentence(td)
#     sentence = sentences[0] #test on one sentence first 
#     print(sentence)
    MASSIVE_YSTAR=[]
    for sentence in sentences:
        betaBase = beta_base(transition,emission,sentence) #get the beta base 
        forward = {} 
        forward[1] = alpha_base #this should be constant
        YSTAR = [] 
        #now do forward 
        for j in range(1,len(sentence)-1):
    #         print(sentence[j])
            temp_alphas = {}
            for u in TAGS:
                alpha_j_1_v = 0.0 #value of alpha u (j+1) 
                for v in TAGS:
                    a = 0.0 
                    b = 0.0 
                    if (v,u) in transition:
                        a = transition[(v,u)]
                    if (sentence[j],v) in emission:
                        b = emission[(sentence[j],v)]
                    alpha_j_1_v += float(forward[j][v] * a * b )
                temp_alphas[u] = alpha_j_1_v
            forward[j+1]= temp_alphas

        #now do backward
        backwards = {}
        backwards[len(sentence)-1] = betaBase # inserting beta u n
        for j in range(len(sentence)-2,0,-1): #j = n-1 ,...,1 
            temp_beta={}
            for u in TAGS: 
                beta_u_j = 0.0 
                for v in TAGS:
                    a = 0.0 
                    b = 0.0 
                    if (u,v) in transition:
                        a = transition[(u,v)]
                    if (sentence[j],u) in emission:
                        b = emission[(sentence[j],u)]
                    beta_u_j += float(backwards[j+1][v]*a*b)
                temp_beta[u] = beta_u_j
            backwards[j] = temp_beta

        #find the tags
        YSTAR.append("START")
        for j in range(1,len(sentence)-1):
            temp = {}
            for u in TAGS:
                temp[u] = forward[j][u] * backwards[j][u]
            YSTAR.append(max(temp,key=temp.get))
        YSTAR.append("STOP")
        MASSIVE_YSTAR.append(YSTAR)
    MASSIVE_YSTAR = process_viterbi_results(MASSIVE_YSTAR,sentences)
    return MASSIVE_YSTAR

## Max Marginal Decoding Try 2

In [136]:
def max_marginal(sentence):
#     sentence.append()
    #sentence is a list containing the words
    # this is only for one sentence
    opti_path = []
    all_tags=["O","B-negative","B-positive","B-neutral","I-negative","I-positive","I-neutral"]
    # forward part
    forward = {}
    alpha_base = {}
    for u in all_tags:
        if giantTransitionDict.get("START" + " " + u) != None:
            alpha_base[u] = giantTransitionDict["START" + " " + u]
        else:
            alpha_base[u] = 0.0
    forward[1] = alpha_base
    for i in range(1,len(sentence)-1):
        if sentence[i] in modifiedWordDict:
            sentence[i] = sentence[i]
        else:
            sentence[i] = "#UNK#"
            
        tempAlpha = {}
        for u in all_tags:
            alpha = 0.0
            for v in all_tags:
                a = 0.0
                b = 0.0
                if giantTransitionDict.get(v + " " + u) != None:
                    a = giantTransitionDict[v + " " + u]
                if giantEmissionDict.get(sentence[i] + " " + v) != None:
                    b = giantEmissionDict[sentence[i] + " " + v]
                alpha += float(forward[i][v] * a * b)
            tempAlpha[u] = alpha
        forward[i + 1] = tempAlpha
    
    
    # backward part
    lastWord = sentence[len(sentence)-1] #last word 
    if lastWord in modifiedWordDict:
        lastWord = lastWord
    else:
        lastWord = "#UNK#"
    backwards = {} 
    beta_base = {}
    for u in all_tags:
        transition_v = 0.0 
        emission_v = 0.0 
        if giantTransitionDict.get(u + " " + "STOP") != None:
            transition_v = giantTransitionDict[u + " " + "STOP"]
        if giantEmissionDict.get(lastWord + " " + u) != None:
            emission_v = giantEmissionDict[lastWord + " " + u]
        beta_base[u] = float(transition_v * emission_v)
    backwards[len(sentence)-1] = beta_base
    for i in range(len(sentence)-2, 0, -1): #n-1, n-2 .... 1
        if sentence[i] in modifiedWordDict:
            sentence[i] = sentence[i]
        else:
            sentence[i] = "#UNK#"
            
        tempBeta = {}
        for u in all_tags:
            beta = 0.0
            for v in all_tags:
                a = 0.0
                b = 0.0
#                 print("POTATOES")
#                 print(giantEmissionDict["potatoes I-positive"])
#                 print(giantTransitionDict["I-positive I-positive"])
                if giantTransitionDict.get(u + " " + v) != None:
                    a = giantTransitionDict[u + " " + v]
                if giantEmissionDict.get(sentence[i] + " " + u) != None:
                    b = giantEmissionDict[sentence[i] + " " + u]
#                 print(backwards[i+1])
                beta += float(backwards[i+1][v]*a*b)                
            tempBeta[u] = beta
        backwards[i] = tempBeta
    print(forward)
    print(backwards)
    for j in range(1,len(sentence)):
        temp = {}
        for u in all_tags:
            temp[u] = forward[j][u] * backwards[j][u]
        opti_path.append(max(temp,key=temp.get))
    print(opti_path)
    return opti_path

max_marginal(["The", "tuna", "and", "wasabe", "potatoes", "are", "excellent", "."])
# for sentence in sentences:
#     max_marginal(sentence)

{1: {'O': 0.9401709401709402, 'B-negative': 0.010683760683760684, 'B-positive': 0.0438034188034188, 'B-neutral': 0.005341880341880342, 'I-negative': 0.0, 'I-positive': 0.0, 'I-neutral': 0.0}, 2: {'O': 0.00016041691565557236, 'B-negative': 1.1555435393460846e-06, 'B-positive': 3.604271538237371e-06, 'B-neutral': 1.7605233978956968e-07, 'I-negative': 1.1745075025777505e-05, 'I-positive': 2.1612581094621358e-05, 'I-neutral': 0.0}, 3: {'O': 4.8999089587278935e-06, 'B-negative': 7.541558282861202e-08, 'B-positive': 2.3522976804713884e-07, 'B-neutral': 1.1489908741201277e-08, 'I-negative': 1.4076295468736675e-07, 'I-positive': 2.3181748487835117e-07, 'I-neutral': 0.0}, 4: {'O': 6.113739625514341e-07, 'B-negative': 8.503587755151126e-09, 'B-positive': 2.6523655989751155e-08, 'B-neutral': 1.2955604613111133e-09, 'I-negative': 1.724144909916714e-08, 'I-positive': 4.979362996071245e-08, 'I-neutral': 3.8888921893296635e-10}, 5: {'O': 1.3276712628426212e-10, 'B-negative': 3.7571318276561267e-13, '

['B-positive', 'I-positive', 'I-positive', 'I-positive', 'O', 'O', 'O']